In [1]:
import json
from imslp_scraping import get_all_composer_pieces, get_composer_url
from pieces import create_piece
from ydata_profiling import ProfileReport
from typing import List, Dict, Any
import pyarrow
import datetime
import pandas as pd
import numpy as np
import polars as pl

from pathlib import Path
file = Path("full_df.parquet")
df = pl.read_parquet(file)
filtered_df = (
    df.group_by('composer_name')
    .agg(pl.count('work_name').alias('count'))
    .filter(pl.col('count') > 10)
    .join(df, on='composer_name')
)
instrument_mapping = {
    'viol': 'violin',
    'piano)': 'piano',
    '(piano': 'piano',
    'Piano Solo': 'piano',
    '1 piano': 'piano',
    'piano (no.12 = 2 voices': 'piano',
    'piano (or harp (no.7 only)': 'piano',
    'piano (nos.5-7)': 'piano',
    'piano (arranged)': 'piano',
    '2 horns)': '2 horns',    
}

filtered_df = filtered_df.with_columns(
    pl.col('instrumentation')
    .list.eval(
        pl.when(pl.element().str.to_lowercase().is_in(instrument_mapping.keys()))
        .then(pl.element().str.to_lowercase().replace(instrument_mapping))
        .otherwise(pl.element().str.to_lowercase())
    )
    .alias('instrumentation')
)
unique_instruments = (
    filtered_df
    .select('instrumentation', 'work_name', 'composer_name', 'imslp_url')
    .explode('instrumentation')
    .unique()
    .sort(by="work_name")
)

/Users/michaelbrandt/personal/PracticeLogger/data_ingest/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from supabase_database import SupabaseDatabase
db = SupabaseDatabase()
try:
    successful, failed = db.bulk_insert_from_df(df)
    print(f"Successfully inserted {successful} pieces")
    if failed:
        print("Failed inserts:")
        for work_name, error in failed:
            print(f"- {work_name}: {error}")
finally:
    db.close()

Successfully inserted 14376 pieces
Failed inserts:
- Weimarer (Gothäer) Passion: invalid input value for enum catalogue_type: "bc"
LINE 9: ...       1, 'Weimarer (Gothäer) Passion', 'BC D 1', 'bc', NULL...
                                                             ^

- Partita: invalid input value for enum catalogue_type: "krebs-wv"
LINE 9:             20, 'Partita', 'Krebs-WV 825', 'krebs-wv', 825, ...
                                                   ^

- O Mensch bewein dein Sünde groß: invalid input value for enum catalogue_type: "p"
LINE 9: ...O Mensch bewein dein Sünde groß', 'P.396 ; T.61', 'p', 396, ...
                                                             ^

- Laß, Fürstin, laß noch einen Strahl: value too long for type character varying(255)

- Fantasia: invalid input value for enum catalogue_type: "kv"
LINE 9:             49, 'Fantasia', 'KV 616a', 'kv', 616, NULL, NULL...
                                               ^

- Horn Concerto: invalid input value for en